In [96]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder ,OneHotEncoder
import pickle

In [97]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [98]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [99]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [100]:
OneHotEncoder_geo = OneHotEncoder()
geo_encoder = OneHotEncoder_geo.fit_transform(data[['Geography']])
Geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns= OneHotEncoder_geo.get_feature_names_out(['Geography']))
Geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [101]:
data = pd.concat([data.drop('Geography',axis=1), Geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [102]:
X = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [104]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('OneHotEncoder_geo.pkl', 'wb') as file:
    pickle.dump(OneHotEncoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)
    

# ANN Regression


In [105]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [106]:
#build the model
model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

#compile the model
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mae'])
model.summary()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [107]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

#setup tensorboard
log_dir = "regressionlogs/fit/"+ datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
TensorBoard_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)


In [108]:
earlystopping_callback = EarlyStopping(monitor = 'val_loss', patience=10, restore_best_weights=True )

In [109]:
#train the model
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=100, 
    callbacks = [earlystopping_callback, TensorBoard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - loss: 101500.6562 - mae: 101500.6562 - val_loss: 98535.0391 - val_mae: 98535.0391
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 99794.4297 - mae: 99794.4297 - val_loss: 97156.2500 - val_mae: 97156.2500
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 98066.0625 - mae: 98066.0625 - val_loss: 93621.0000 - val_mae: 93621.0000
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 93314.0859 - mae: 93314.0859 - val_loss: 87424.9141 - val_mae: 87424.9141
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 87295.3906 - mae: 87295.3906 - val_loss: 79099.4922 - val_mae: 79099.4922
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - loss: 78219.7500 - mae: 78219.7500 - val_loss: 70120.3828 - val_mae: 70120.3828
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 69153.7031 - mae: 69153.7031 - val_loss: 62260.0273 - val_mae: 62260.0273
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━

In [110]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [111]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6009 (pid 47477), started 3:39:22 ago. (Use '!kill 47477' to kill it.)

In [112]:
test_loss, test_mae = model.evaluate(X_test ,y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 272us/step - loss: 51117.1719 - mae: 51117.1719
Test MAE: 50308.984375


In [113]:
model.save('Regression_model.keras')